In [1]:
! pip install -U wandb
! pip install evaluate
! pip install peft==0.12.0
! pip install -U bitsandbytes
! pip install sacrebleu
! pip install -U transformers
! pip install datasets
! pip install rouge_score

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 1.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.0/457.0 kB 1.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 1.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: wandb
    Found existing installation: wandb 0.18.7
    Uninstalling wandb-0.18.7:
      Successfully uninstalled wandb-0.18.7
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.3.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://mirrors.aliyun.com/pypi/simple/

[notice] A new release of pip is available: 23.3.2 -> 24.3.1
[no

In [2]:
import os
os.environ["HF_ENDPOINT"]="https://hf-mirror.com"

from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, \
    Seq2SeqTrainer
import evaluate
import numpy as np
import wandb
from  datasets import Dataset
from concurrent.futures import ProcessPoolExecutor


dataset = load_dataset("fixed_train") 

print(dataset)
# 32962
train_val = dataset['train'].train_test_split(test_size=0.1)
train_data = train_val['train']
print(train_data)
# 29665
val_data = train_val['test']
print(val_data)
# 3297

/home/pod/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using the latest cached version of the dataset since fixed_train couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /home/pod/.cache/huggingface/datasets/fixed_train/default/0.0.0/b456ab493c963d98 (last modified on Tue Dec  3 09:32:09 2024).


DatasetDict({
    train: Dataset({
        features: ['id', 'source_locale', 'target_locale', 'source', 'target', 'entities', 'from'],
        num_rows: 32962
    })
})
Dataset({
    features: ['id', 'source_locale', 'target_locale', 'source', 'target', 'entities', 'from'],
    num_rows: 29665
})
Dataset({
    features: ['id', 'source_locale', 'target_locale', 'source', 'target', 'entities', 'from'],
    num_rows: 3297
})


In [3]:
if __name__=="__main__":
    os.environ['WANDB_API_KEY'] = "a464ce6c3b972e3e7090ac20839b9a1daac1b608"
    os.environ["WANDB_PROJECT"] = "flan-T5-large-max_len=64"
#     wandb.init()
    
    # 尝试移除无用列
    column_name = train_data.column_names

    model_id = "google/flan-t5-large"

    tokenizer = AutoTokenizer.from_pretrained(model_id)

#     prefix = '''Given a input sentence from the source language (en) that contains named entities, your task is to accurately translate the named entities in the input sentence to the target language {}: '''
    prefix = "translate English to {}: "
    # metrics
    bleuM = evaluate.load("sacrebleu")
    rougeM = evaluate.load("rouge")
    
    model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
    # 使用data_collator将数据集中的多个样本合并成一个batch，对齐输入和标签的长度
    data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer,model=model)
    def preprocess_function(examples):
#         '''
#         在需要处理一个批次样本时，可以通过设置 batched=True，让 map 传入一组样本的批次。
#         此时，自定义函数的参数会变成一个包含每列字段的字典，字段的值为一个列表，代表该批次中所有样本在对应列下的数据。
#         这样，用户可以灵活选择是否逐条或批量处理数据。
#         :param examples:
#         :return:
#         '''
#         inputs = [prefix.format(target_lcoale)+example for example,target_lcoale in zip(examples['source'],examples['target_locale'])]
        translate_dict={
            "ar":"Arabic",
            "de":"German",
            "es":"Spanish",
            "fr":"French",
            "it":"Italian",
            "ja":"Japanese",
        }
        inputs = [prefix.format(translate_dict[target_lcoale])+example for example,target_lcoale in zip(examples['source'],examples['target_locale'])]
        
        
        targets = examples['target']
        
        
#         model_inputs = tokenizer(inputs,text_target=targets,max_length=50,truncation=True,padding=True,return_tensors="pt")

        model_inputs = tokenizer(inputs, max_length=64, truncation=True, padding="max_length", return_tensors='pt')
        labels = tokenizer(targets, max_length=64, truncation=True, padding="max_length",return_tensors='pt')

        # 将目标文本的填充 token 设置为 -100，以便在计算损失时忽略
        labels["input_ids"] = [
            [l if l != tokenizer.pad_token_id else -100 for l in label] for label in labels["input_ids"]
        ]
        # 将编码后的目标设置为模型的标签
        model_inputs["labels"] = labels["input_ids"]
    
       
        return model_inputs

In [4]:
print(train_data)
# 29665
print(train_data[0])

Dataset({
    features: ['id', 'source_locale', 'target_locale', 'source', 'target', 'entities', 'from'],
    num_rows: 29665
})
{'id': '71a173fc', 'source_locale': 'en', 'target_locale': 'es', 'source': 'When did the Saxons invade England?', 'target': '¿Cuándo invadieron Inglaterra los sajones?', 'entities': ['Q21'], 'from': 'mintaka'}


In [5]:
    tokenized_train = train_data.map(preprocess_function,batched=True,remove_columns=column_name)
    tokenized_val = val_data.map(preprocess_function,batched=True,remove_columns=column_name)
    # 使用一部分进行验证
#     tokenized_val_slice = Dataset.from_dict(tokenized_val[0:1000])

Map: 100%|██████████| 3297/3297 [00:04<00:00, 820.60 examples/s]


In [6]:
    def postprocess_text(preds, labels):
        preds = [pred.strip() for pred in preds]
        labels = [[label.strip()] for label in labels]
        return preds, labels
    
    
    def compute_metrics(eval_pred):
        preds, labels = eval_pred
        
        if isinstance(preds, tuple):
            preds = preds[0]
            
#         print(preds.shape)
#         batchsize,seq_len,vocab_size
#         (100, 64, 32128)
#         input()
        
        # 将 predictions 从概率转为 token IDs ?
        preds = np.argmax(preds,axis=-1) # (batch_size, seq_len)
        
        # 需要额外处理label吗？
        # transformers 中，填充位置通常被标记为 -100，需要将这些位置替换为 tokenizer.pad_token_id，避免在解码时生成错误文本
        
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        
        decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
        decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
     
        # 已经是批量了
#         result1 = bleuM.compute(predictions=decoded_preds, references=decoded_labels)
#         result2 = rougeM.compute(predictions=decoded_preds,references=decoded_labels)
        
        
        result = rougeM.compute(predictions=decoded_preds,references=decoded_labels)
        reuslt_bleu = bleuM.compute(predictions=decoded_preds,references=decoded_labels)
        result['bleu'] = reuslt_bleu['score']
        
#         result = {
#             "bleu": result1["score"],
#             "rouge":result2['score'],
#         }
        
        prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
        result["gen_len"] = np.mean(prediction_lens)
        
        # 下面这一步好像没法批量进行，导致GPU占用归零，计算时间非常长
#         result = {k: round(v, 4) for k, v in result.items()}
        return result
    


In [ ]:
    #     tokenized_val_slice = Dataset.from_dict(tokenized_val[0:100])
    #     tokenized_train_slice = Dataset.from_dict(tokenized_train[0:30])
    # 定义训练的超参数
    training_args = Seq2SeqTrainingArguments(
        output_dir='./semeval-flan-T5-large-max_len=64',  # output directory
        save_total_limit=3,       # 只保留最近的 3 个检查点
#         report_to="wandb",
        num_train_epochs=3,  # total number of training epochs
        per_device_train_batch_size=2,  # batch size per device during training
        per_device_eval_batch_size=4,  # batch size for evaluation
        warmup_steps=200,  # number of warmup steps for learning rate scheduler
        weight_decay=0.01,  # strength of weight decay
        logging_dir='./logs',  # directory for storing logs
        
        learning_rate=2e-5,  # Set learning rate
        evaluation_strategy="steps",
        
        
        # 频繁的日志记录可能导致内存增加，特别是当模型需要频繁评估时。
        logging_steps=500,
        eval_accumulation_steps=100,
        
        # 使用T5-base时遇到loss为NAN的问题，或许尝试不使用gradient_checkpointing
        # 尝试禁用gradient_checkpointing和FP16，跑出了结果
        
        # 过多裁剪会导致不能学习到有用的知识
#         max_grad_norm=1.0,  # 最大梯度裁剪，防止梯度爆炸
        
        save_strategy="epoch",
        
        gradient_checkpointing=True,
#         fp16=True, # 开启混合精度
        # gradient_accumulation_steps=8,  # 累积 8 个小批次
    )

    trainer = Seq2SeqTrainer(
        model = model,
        args = training_args,
        train_dataset = tokenized_train,
        eval_dataset = tokenized_val,
        
#         train_dataset = tokenized_train_slice,
#         eval_dataset=tokenized_val_slice,
        
        tokenizer = tokenizer,
        data_collator = data_collator,
        compute_metrics = compute_metrics,
    )

    trainer.train()
#     trainer.evaluate()
    wandb.finish()
    trainer.save_model("./my_model_final-flan-T5-large-max_len=64")

/home/pod/.local/lib/python3.10/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_829/4029350877.py:36: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Unable to read the token file at /var/run/secrets/kubernetes.io/serviceaccount/token due to permission error ([Errno 13] Permission denied: '/var/run/secrets/kubernetes.io/serviceaccount/token').The current user id is 1000. Consider changing the securityContext to run the container as the current user.
wandb: WARNING Unable to read the token 

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Bleu,Gen Len
500,0.962500,0.750431,0.260230,0.156085,0.258091,0.258202,10.779309,64.000000


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


In [ ]:
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, \
    Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained("./my_model_final-flan-T5-large-max_len=64").to("cuda")
tokenizerSelf = AutoTokenizer.from_pretrained("./my_model_final-flan-T5-large-max_len=64")
text = "translate English to Italian: What video game had the most sales in 2006?"
# target：Quale videogioco ha registrato il maggior numero di vendite nel 2006?
inputs = tokenizerSelf(text,return_tensors="pt").to("cuda")
print(inputs)
outputs = model.generate(inputs['input_ids'],max_length=64,num_beams=5,early_stopping=True)

decoded_output = tokenizerSelf.decode(outputs[0], skip_special_tokens=True)
print(decoded_output)

In [ ]:
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, \
    Seq2SeqTrainer
model_id = "google/flan-t5-large"
model = AutoModelForSeq2SeqLM.from_pretrained(model_id).to("cuda")
tokenizer = AutoTokenizer.from_pretrained(model_id)
# text = "translate English to Japanese: Hello, how are you ? "
text = "translate English to Italian: What video game had the most sales in 2006?"
inputs = tokenizer(text,return_tensors="pt").to("cuda")
outputs = model.generate(inputs['input_ids'],max_length=64,num_beams=5,early_stopping=True)

decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(decoded_output)